In [ ]:
# event_collector.py
import numpy as np
import time
import requests
import pandas as pd
import logging
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuração de Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Intervalo de datas
start_date = datetime(2025, 1, 17)
end_date = datetime(2025, 1, 17)

# Detalhes da API
url_events = "https://api.b365api.com/v3/events/ended"
token = "183604-pWN7flhoAsWGu8"

def make_request_with_retry(url, params, retries=5, backoff_factor=2, timeout=10):
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            wait_time = backoff_factor ** attempt
            logging.warning(f"Tentativa {attempt} falhou: {e}. Retentando em {wait_time} segundos.")
            time.sleep(wait_time)
            if attempt == retries:
                logging.error(f"Falha após {retries} tentativas para URL: {url}")
                raise e

def fetch_events_for_date(date):
    formatted_date = date.strftime('%Y%m%d')
    params = {
        "token": token,
        "sport_id": "1",
        "league_id": "22614",
        "day": formatted_date,
        "page": 1
    }
    events = []
    while True:
        try:
            response = make_request_with_retry(url_events, params)
            data = response.json()
            events.extend(data['results'])
            if params['page'] * data['pager']['per_page'] < data['pager']['total']:
                params['page'] += 1
            else:
                break
        except Exception as e:
            logging.error(f"Erro ao buscar eventos para {formatted_date}: {e}")
            break
    return events


In [ ]:
all_events = []
for single_date in pd.date_range(start_date, end_date):
    events = fetch_events_for_date(single_date)
    all_events.extend(events)
    logging.info(f"Eventos coletados em {single_date.strftime('%Y-%m-%d')}: {len(events)}")

In [ ]:
"""All API-related constants."""

API_TOKEN = "183604-pWN7flhoAsWGu8"

SPORT_ID = 1

LEAGUE_IDS = {
   '22614': "Battle 8min"
}

MARKET_IDS = {
    "moneyline": '1_1',
    "handicap": '1_2',
    "goals": '1_3'
}

URLS = {
    "inplay": "https://api.b365api.com/v3/events/inplay",
    "odds": "https://api.b365api.com/v2/event/odds",
    "ended_events": "https://api.b365api.com/v3/events/ended",
    "event": "https://api.b365api.com/v1/event/view"
}


def events_for_date(dates: list, league_ids: dict = LEAGUE_IDS) -> list[dict]:
    
    """Fetch all ended events for a specific list of dates

    Args:
        dates (list): List of dates to fetch events from.
        league_ids (dict, optional): Mapping of league identifiers. 
        Format example: {"soccer": 1, "basketball": 2}. 
        Defaults to LEAGUE_IDS.

    Returns:
        events: list of dicts containing all events.
    
    TODO: add parameter to choose league.
    """
    logging.info('Starting to fetch data')

    league_ids = (LEAGUE_IDS.keys())

    all_events = []

    for date in dates:

        print('inside loop 1')
        formatted_date = date.strftime('%Y%m%d')

        params = {
            "token": API_TOKEN,
            "sport_id": SPORT_ID,
            "league_id": league_ids,  
            "day": formatted_date,
            "page": 1
        }

        
        while True:
            events = []
            response = requests.get(URLS['ended_events'], params=params)
            
            if response.status_code == 200:
                data = response.json()
                events.extend(data['results'])
                if params['page'] * data['pager']['per_page'] < data['pager']['total']:
                    params['page'] += 1
                else:
                    break
            else:
                logging.error(f"Request failed: {response.status_code} for date {formatted_date}")
                break
        
        logging.info(f"Fetched {len(events)} events for date {date.strftime('%Y-%m-%d')}")
        
        all_events.extend(events)

    return all_events

ModuleNotFoundError: No module named 'api'

In [5]:
events_for_date([datetime(2025, 1, 17)])

NameError: name 'events_for_date' is not defined